**This script takes bands of large rasters (stored as numpy arrays) and then splits them into pieces to allow for regression predictions from a random forest model trained and built in a seperate script**

In [ ]:
!apt install gdal-bin python3-gdal --quiet

# Part 1

**This code could be significantly improved, however due to memory limitations the best way I found was to manually run this part for each band of each year's imagery that you were interested in**

Load in a band

In [ ]:
import numpy as np
import joblib

b1 = np.load("")

The next two code blocks are just quick checks to make sure that these arrays can be cleanly split into 124 pieces (landed on 124 after many rounds of trial and error)

In [ ]:
new_shape = (b1.shape[0] * b1.shape[1],1)
temp_array = b1.reshape(new_shape)

In [ ]:
temp_array.shape[0]/124

39579310.0

Now split the array into 124 pieces

In [ ]:
split = np.array_split(temp_array,124,axis=0)

loop through each split and save it as a numpy array

In [ ]:
from tqdm import tqdm
for i in tqdm(range(0,len(split))):
  path = '' + str(i) + '.npy'
  np.save(path,split[i])

100%|██████████| 23/23 [00:30<00:00,  1.34s/it]


# Part 2

Now load in the RF model. Then loop through each round of splits, stitch them together, run them through the model, them save the results as a numpy array

In [ ]:
from tqdm import tqdm
import numpy as np
import joblib
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import ensemble
regr = joblib.load("")


for i in tqdm(range(0,124)):
  b1 = np.load('' + str(i) + '.npy')
  b2 = np.load('' + str(i) + '.npy')
  b3 = np.load('' + str(i) + '.npy')
  b4 = np.load('' + str(i) + '.npy')
  b5 = np.load('' + str(i) + '.npy')
  b6 = np.load('' + str(i) + '.npy')
  test = np.append(b1,b2,axis=1)
  test = np.append(test,b3,axis=1)
  test = np.append(test,b4,axis=1)
  test = np.append(test,b5,axis=1)
  test = np.append(test,b6,axis=1)
  image = test
  image[np.isnan(image)] = 0
  prediction = regr.predict(image)
  path = '' + str(i) + '.npy'
  np.save(path,prediction)

100%|██████████| 124/124 [2:07:54<00:00, 61.89s/it]


Now stitch together the pieces of the predicted results. These have to be stored as two seperate arrays due to size (0-62,62-124)

In [ ]:
import numpy as np
from tqdm import tqdm

# Loop through each file in the folder
for i in tqdm(range(62,124)):
    # Construct the full path to the file
    path = '' + str(i) + '.npy'
    # Load the numpy array from the file
    array = np.load(path,allow_pickle=True)
    # Append the array to the list
    if i == 62:
      result = array
    else:
      result = np.append(result,array,axis=0)
      del array


# Print the shape of the resulting array
print("Shape of the concatenated array:", result.shape)
np.save('',result)


100%|██████████| 62/62 [08:54<00:00,  8.62s/it]


Shape of the concatenated array: (2453917220,)


# Part 3

Now stitch together the two large arrays, reshape them to be the dimensions of a raster, then save them as a numpy array

In [ ]:
import numpy as np
r1 = np.load('',mmap_mode='r')
r2 = np.load('',mmap_mode='r')

r3 = np.append(r1,r2,axis=0)
base =  np.load("",mmap_mode='r')
r4 = r3.reshape(base.shape)
np.save('',r4)

# Part 4

Load in a raster of equal size to the predicted image so that you can easily pull dimensions, reolutions, projection, etc

In [ ]:
from osgeo import gdal
import glob
import subprocess
gdal.UseExceptions()

# list all files in directory that match pattern
demList = glob.glob("*.tif")
print(demList)
first = gdal.Open(demList[0])
img_reference = first.GetGeoTransform()
x_res = img_reference[1]
y_res = -img_reference[5]

#merge
#cmd = "gdal_merge.py -o mergedDEM.tif"
#subprocess.call(cmd.split()+demList)
options_list = [
    '-hidenodata'
]
options_string = " ".join(options_list)

# build virtual raster and convert to geotiff
vrt = gdal.BuildVRT("MERGE.vrt", demList,options='-hidenodata -srcnodata "0"')
vrt = None

['/content/drive/MyDrive/2023_LS_NOV3try/2023_LS_Nov3-0000000000-0000000000.tif', '/content/drive/MyDrive/2023_LS_NOV3try/2023_LS_Nov3-0000000000-0000009472.tif', '/content/drive/MyDrive/2023_LS_NOV3try/2023_LS_Nov3-0000000000-0000018944.tif', '/content/drive/MyDrive/2023_LS_NOV3try/2023_LS_Nov3-0000000000-0000028416.tif', '/content/drive/MyDrive/2023_LS_NOV3try/2023_LS_Nov3-0000000000-0000037888.tif', '/content/drive/MyDrive/2023_LS_NOV3try/2023_LS_Nov3-0000000000-0000047360.tif', '/content/drive/MyDrive/2023_LS_NOV3try/2023_LS_Nov3-0000000000-0000056832.tif', '/content/drive/MyDrive/2023_LS_NOV3try/2023_LS_Nov3-0000000000-0000066304.tif', '/content/drive/MyDrive/2023_LS_NOV3try/2023_LS_Nov3-0000000000-0000075776.tif', '/content/drive/MyDrive/2023_LS_NOV3try/2023_LS_Nov3-0000009472-0000000000.tif', '/content/drive/MyDrive/2023_LS_NOV3try/2023_LS_Nov3-0000009472-0000009472.tif', '/content/drive/MyDrive/2023_LS_NOV3try/2023_LS_Nov3-0000009472-0000018944.tif', '/content/drive/MyDrive/202

Now save the stitched together and reshped array as a tif. Make sure to update the SetNoDataValue() parameter for each run

In [ ]:
from osgeo import gdal,gdal_array
gdal.UseExceptions()
gdal.AllRegister()


in_file = r"/content/MERGE.vrt"
img_ds = gdal.Open(in_file)
driver = gdal.GetDriverByName('GTiff')
outDs = driver.Create("", img_ds.RasterXSize, img_ds.RasterYSize, 1, gdal.GDT_Float32)
outBand = outDs.GetRasterBand(1)
outBand.SetNoDataValue(0.29330593)
outBand.WriteArray(r4)
outDs.SetGeoTransform(img_ds.GetGeoTransform())

0